# <font color=Blue> Extended Atom-Type Geometric Graph Learning Example </font>

In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from rdkit import Chem
from scipy.spatial.distance import cdist
from itertools import product
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from biopandas.pdb import PandasPdb
from biopandas.mol2 import PandasMol2

RDKit WARNING: [22:40:36] Enabling RDKit 2019.09.3 jupyter extensions


## <font color=Blue> Define a class to build the kernel function </font>

In [2]:
class KernelFunction:

    def __init__(self, kernel_type='exponential_kernel',
                 kappa=2.0, tau=1.0):
        self.kernel_type = kernel_type
        self.kappa = kappa
        self.tau = tau

        self.kernel_function = self.build_kernel_function(kernel_type)

    def build_kernel_function(self, kernel_type):
        if kernel_type[0] in ['E', 'e']:
            return self.exponential_kernel
        elif kernel_type[0] in ['L', 'l']:
            return self.lorentz_kernel

    def exponential_kernel(self, d, vdw_radii):
        eta = self.tau*vdw_radii

        return np.exp(-(d/eta)**self.kappa)

    def lorentz_kernel(self, d, vdw_radii):
        eta = self.tau*vdw_radii

        return 1/(1+(d/eta)**self.kappa)

In [3]:
# Build the kernel from a given kernel type with parameters $\kappa$ and $\tau$
Kernel = KernelFunction(kernel_type='exponential_kernel',
                 kappa=6, tau=4)

## <font color=Blue> Define a class for the SYBYL Atom-Type GGL model </font>

In [4]:
class SYBYL_GGL:

    protein_atom_types_df = pd.read_csv(
        '../utils/protein_atom_types.csv')

    ligand_atom_types_df = pd.read_csv(
        '../utils/ligand_SYBYL_atom_types.csv')

    protein_atom_types = protein_atom_types_df['AtomType'].tolist()
    protein_atom_radii = protein_atom_types_df['Radius'].tolist()

    ligand_atom_types = ligand_atom_types_df['AtomType'].tolist()
    ligand_atom_radii = ligand_atom_types_df['Radius'].tolist()

    protein_ligand_atom_types = [
        i[0]+"-"+i[1] for i in product(protein_atom_types, ligand_atom_types)]

    def __init__(self, Kernel, cutoff=12.0):

        self.Kernel = Kernel
        self.cutoff = cutoff

        self.pairwise_atom_type_radii = self.get_pairwise_atom_type_radii()

    def get_pairwise_atom_type_radii(self):

        protein_atom_radii_dict = {a: r for (a, r) in zip(
            self.protein_atom_types, self.protein_atom_radii)}

        ligand_atom_radii_dict = {a: r for (a, r) in zip(
            self.ligand_atom_types, self.ligand_atom_radii)}

        pairwise_atom_type_radii = {i[0]+"-"+i[1]: protein_atom_radii_dict[i[0]] +
                                    ligand_atom_radii_dict[i[1]] for i in product(self.protein_atom_types, self.ligand_atom_types)}

        return pairwise_atom_type_radii

    def mol2_to_df(self, mol2_file):
        df_mol2 = PandasMol2().read_mol2(mol2_file).df
        df = pd.DataFrame(data={'ATOM_INDEX': df_mol2['atom_id'],
                                'ATOM_ELEMENT': df_mol2['atom_type'],
                                'X': df_mol2['x'],
                                'Y': df_mol2['y'],
                                'Z': df_mol2['z']})

        if len(set(df["ATOM_ELEMENT"]) - set(self.ligand_atom_types)) > 0:
            print(
                "WARNING: Ligand contains unsupported atom types. Only supported atom-type pairs are counted.")
        return(df)

    def pdb_to_df(self, pdb_file):
        ppdb = PandasPdb()
        ppdb.read_pdb(pdb_file)
        ppdb_all_df = ppdb.df['ATOM']
        ppdb_df = ppdb_all_df[ppdb_all_df['atom_name'].isin(
            self.protein_atom_types)]
        atom_index = ppdb_df['atom_number']
        atom_element = ppdb_df['atom_name']
        x, y, z = ppdb_df['x_coord'], ppdb_df['y_coord'], ppdb_df['z_coord']
        df = pd.DataFrame.from_dict({'ATOM_INDEX': atom_index, 'ATOM_ELEMENT': atom_element,
                                     'X': x, 'Y': y, 'Z': z})

        return df

    def get_mwcg_rigidity(self, protein_file, ligand_file):
        '''
            Adapted from ECIF package
        '''
        protein = self.pdb_to_df(protein_file)
        ligand = self.mol2_to_df(ligand_file)

        # select protein atoms in a cubic with a size of cutoff from ligand
        for i in ["X", "Y", "Z"]:
            protein = protein[protein[i] < float(ligand[i].max())+self.cutoff]
            protein = protein[protein[i] > float(ligand[i].min())-self.cutoff]

        atom_pairs = list(
            product(protein["ATOM_ELEMENT"], ligand["ATOM_ELEMENT"]))
        atom_pairs = [x[0]+"-"+x[1] for x in atom_pairs]
        pairwise_radii = [self.pairwise_atom_type_radii[x]
                          for x in atom_pairs]
        pairwise_radii = np.asarray(pairwise_radii)

        pairwise_mwcg = pd.DataFrame(atom_pairs, columns=["ATOM_PAIR"])
        distances = cdist(protein[["X", "Y", "Z"]],
                          ligand[["X", "Y", "Z"]], metric="euclidean")
        pairwise_radii = pairwise_radii.reshape(
            distances.shape[0], distances.shape[1])
        mwcg_distances = self.Kernel.kernel_function(distances, pairwise_radii)

        distances = distances.ravel()
        mwcg_distances = mwcg_distances.ravel()
        mwcg_distances = pd.DataFrame(
            data={"DISTANCE": distances, "MWCG_DISTANCE": mwcg_distances})
        pairwise_mwcg = pd.concat([pairwise_mwcg, mwcg_distances], axis=1)
        pairwise_mwcg = pairwise_mwcg[pairwise_mwcg["DISTANCE"] <= self.cutoff].reset_index(
            drop=True)

        return pairwise_mwcg

    def get_ggl_score(self, protein_file, ligand_file):
        features = ['COUNTS', 'SUM', 'MEAN', 'STD', 'MIN', 'MAX']
        pairwise_mwcg = self.get_mwcg_rigidity(protein_file, ligand_file)
        mwcg_temp_grouped = pairwise_mwcg.groupby('ATOM_PAIR')
        mwcg_temp_grouped.agg(['sum', 'mean', 'std', 'min', 'max'])
        mwcg_temp = mwcg_temp_grouped.size().to_frame(name='COUNTS')
        mwcg_temp = (mwcg_temp
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'sum'}).rename(columns={'MWCG_DISTANCE': 'SUM'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'mean'}).rename(columns={'MWCG_DISTANCE': 'MEAN'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'std'}).rename(columns={'MWCG_DISTANCE': 'STD'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'min'}).rename(columns={'MWCG_DISTANCE': 'MIN'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'max'}).rename(columns={'MWCG_DISTANCE': 'MAX'}))
                     )
        mwcg_columns = {'ATOM_PAIR': self.protein_ligand_atom_types}
        for _f in features:
            mwcg_columns[_f] = np.zeros(len(self.protein_ligand_atom_types))
        ggl_score = pd.DataFrame(data=mwcg_columns)
        ggl_score = ggl_score.set_index('ATOM_PAIR').add(
            mwcg_temp, fill_value=0).reindex(self.protein_ligand_atom_types).reset_index()

        return ggl_score

In [5]:
# Construct the SYBYL atom-type GGL class with the given Kernel and a cutoff distance
sybyl_GGL = SYBYL_GGL(Kernel, cutoff=12.0)

In [6]:
# Path for the protein pdb file and ligand mol2 file of the example structure
protein_file = 'Example_Complex/1a1e/1a1e_protein.pdb'
ligand_file = 'Example_Complex/1a1e/1a1e_ligand.mol2'

In [7]:
protein_df = sybyl_GGL.pdb_to_df(protein_file)
protein_df.head()

,ATOM_INDEX,ATOM_ELEMENT,X,Y,Z
0,1,N,57.904,24.527,16.458
4,5,CA,56.635,24.239,17.114
6,7,C,56.572,24.562,18.605
7,8,O,55.469,24.718,19.110
8,9,CB,56.298,22.729,16.823


In [8]:
ligand_df = sybyl_GGL.mol2_to_df(ligand_file)
ligand_df.head()

,ATOM_INDEX,ATOM_ELEMENT,X,Y,Z
0,1,C.2,43.293,-8.128,37.550
1,2,O.2,44.067,-7.186,37.788
2,3,C.3,43.519,-9.010,36.332
3,4,N.am,42.169,-8.311,38.208
4,5,C.3,41.875,-7.504,39.354


In [9]:
# Pairwise protein-ligand SYBYL atom type colored graph for the example structure
pairwise_mwcg = sybyl_GGL.get_mwcg_rigidity(protein_file, ligand_file)
pairwise_mwcg.head()

,ATOM_PAIR,DISTANCE,MWCG_DISTANCE
0,CE1-O.co2,11.748670,0.562132
1,CZ-O.co2,11.963535,0.526140
2,OH-O.co2,11.370757,0.512450
3,C-O.co2,11.510658,0.600823
4,O-O.co2,11.338683,0.518240


In [10]:
# SYBYL-GGL features for the example structure
sybyl_ggl_score = sybyl_GGL.get_ggl_score(protein_file, ligand_file)
sybyl_ggl_score.head(20)

,ATOM_PAIR,COUNTS,SUM,MEAN,STD,MIN,MAX
0,C-As,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,C-B,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,C-Be,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,C-Br,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,C-C.1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,C-C.2,64.0,52.912361,0.826756,0.120126,0.625105,0.998047
6,C-C.3,236.0,200.042887,0.847639,0.113318,0.625193,0.999494
7,C-C.ar,151.0,131.640950,0.871794,0.114727,0.635393,0.999541
8,C-C.cat,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,C-Cl,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


## <font color=Blue> Define a class for the ECIF Atom-Type GGL model </font>

In [11]:
class ECIF_GGL:
    # Possible predefined protein atoms
    ECIF_ProteinAtoms = ['C;4;1;3;0;0', 'C;4;2;1;1;1', 'C;4;2;2;0;0', 'C;4;2;2;0;1',
                         'C;4;3;0;0;0', 'C;4;3;0;1;1', 'C;4;3;1;0;0', 'C;4;3;1;0;1',
                         'C;5;3;0;0;0', 'C;6;3;0;0;0', 'N;3;1;2;0;0', 'N;3;2;0;1;1',
                         'N;3;2;1;0;0', 'N;3;2;1;1;1', 'N;3;3;0;0;1', 'N;4;1;2;0;0',
                         'N;4;1;3;0;0', 'N;4;2;1;0;0', 'O;2;1;0;0;0', 'O;2;1;1;0;0',
                         'S;2;1;1;0;0', 'S;2;2;0;0;0']
    # Possible ligand atoms according to the PDBbind 2016 "general set"
    ECIF_LigandAtoms = ['As;1;1;0;0;0', 'As;4;4;0;0;0', 'B;1;1;0;0;0', 'B;2;2;0;0;0',
                        'B;3;2;1;0;0', 'B;3;3;0;0;0', 'B;3;3;0;0;1', 'B;3;4;0;0;0',
                        'B;3;4;0;0;1', 'B;4;4;0;0;0', 'B;5;4;0;0;0', 'B;5;5;0;0;0',
                        'Be;4;4;0;0;0', 'Br;1;1;0;0;0', 'Br;4;4;0;0;0', 'C;1;1;0;0;0',
                        'C;2;2;0;0;0', 'C;2;2;0;0;1', 'C;3;2;0;0;0', 'C;3;2;0;0;1',
                        'C;3;2;0;1;1', 'C;3;2;1;0;1', 'C;3;2;2;0;0', 'C;3;3;0;0;0',
                        'C;3;3;0;0;1', 'C;3;3;0;1;1', 'C;3;3;1;0;1', 'C;4;1;1;0;0',
                        'C;4;1;2;0;0', 'C;4;1;3;0;0', 'C;4;2;0;0;0', 'C;4;2;0;0;1',
                        'C;4;2;1;0;0', 'C;4;2;1;0;1', 'C;4;2;1;1;1', 'C;4;2;2;0;0',
                        'C;4;2;2;0;1', 'C;4;3;0;0;0', 'C;4;3;0;0;1', 'C;4;3;0;1;1',
                        'C;4;3;1;0;0', 'C;4;3;1;0;1', 'C;4;3;1;1;1', 'C;4;4;0;0;0',
                        'C;4;4;0;0;1', 'C;4;4;0;1;1', 'C;5;2;0;0;0', 'C;5;2;1;0;1',
                        'C;5;2;2;0;0', 'C;5;2;3;0;0', 'C;5;3;0;0;0', 'C;5;3;0;0;1',
                        'C;5;3;0;1;1', 'C;5;3;1;0;1', 'C;5;3;2;0;0', 'C;5;3;2;0;1',
                        'C;5;4;1;0;1', 'C;6;3;0;0;0', 'C;6;6;0;0;0', 'Cl;1;1;0;0;0',
                        'Cl;2;1;1;0;0', 'Cl;2;2;0;0;1', 'Cl;3;3;0;0;1', 'Cu;2;2;0;0;1',
                        'Cu;4;4;0;0;1', 'F;1;1;0;0;0', 'F;2;1;1;0;0', 'Fe;4;4;0;0;1',
                        'Fe;5;5;0;0;1', 'Fe;6;4;2;0;1', 'Fe;6;5;1;0;1', 'Fe;6;6;0;0;1',
                        'Fe;9;9;0;0;1', 'Hg;1;1;0;0;0', 'I;1;1;0;0;0', 'Ir;3;4;0;0;1',
                        'Mg;4;4;0;0;1', 'N;2;1;1;0;0', 'N;2;2;0;0;0', 'N;2;2;0;0;1',
                        'N;3;1;0;0;0', 'N;3;1;1;0;0', 'N;3;1;2;0;0', 'N;3;2;0;0;0',
                        'N;3;2;0;0;1', 'N;3;2;0;1;1', 'N;3;2;1;0;0', 'N;3;2;1;0;1',
                        'N;3;2;1;1;1', 'N;3;2;2;0;1', 'N;3;3;0;0;0', 'N;3;3;0;0;1',
                        'N;3;3;0;1;1', 'N;4;1;2;0;0', 'N;4;1;3;0;0', 'N;4;2;0;0;0',
                        'N;4;2;0;0;1', 'N;4;2;1;0;0', 'N;4;2;1;0;1', 'N;4;2;2;0;0',
                        'N;4;2;2;0;1', 'N;4;3;0;0;0', 'N;4;3;0;0;1', 'N;4;3;1;0;0',
                        'N;4;3;1;0;1', 'N;4;4;0;0;0', 'N;4;4;0;0;1', 'N;5;2;0;0;0',
                        'N;5;2;1;0;0', 'N;5;3;0;0;0', 'N;5;3;0;1;1', 'N;5;4;0;0;0',
                        'O;1;1;0;0;0', 'O;1;1;1;0;0', 'O;1;2;0;0;1', 'O;2;0;2;0;0',
                        'O;2;1;0;0;0', 'O;2;1;1;0;0', 'O;2;2;0;0;0', 'O;2;2;0;0;1',
                        'O;2;2;0;1;1', 'O;3;1;0;0;0', 'O;3;1;1;0;0', 'O;3;1;2;0;0',
                        'O;3;2;0;0;1', 'O;3;2;1;0;0', 'O;3;2;1;0;1', 'Os;8;8;0;0;1',
                        'P;4;3;0;0;0', 'P;4;4;0;0;0', 'P;5;3;0;0;0', 'P;5;3;1;0;0',
                        'P;5;4;0;0;0', 'P;5;4;0;0;1', 'P;6;3;1;0;0', 'P;6;4;0;0;0',
                        'P;6;4;0;0;1', 'P;7;4;0;0;0', 'Pt;0;2;0;0;1', 'Pt;3;3;0;0;1',
                        'Re;8;8;0;0;1', 'Rh;1;3;0;0;1', 'Rh;9;9;0;0;1', 'Ru;6;6;0;0;1',
                        'Ru;7;7;0;0;1', 'Ru;8;8;0;0;1', 'Ru;9;9;0;0;1', 'S;2;1;0;0;0',
                        'S;2;1;1;0;0', 'S;2;2;0;0;0', 'S;2;2;0;0;1', 'S;3;2;0;0;1',
                        'S;3;3;0;0;0', 'S;3;3;0;0;1', 'S;4;3;0;0;0', 'S;5;3;0;0;0',
                        'S;5;4;0;0;0', 'S;6;3;1;0;0', 'S;6;4;0;0;0', 'S;6;4;0;0;1',
                        'S;7;4;0;0;0', 'Sb;7;4;0;0;0', 'Se;1;1;0;0;0', 'Se;2;2;0;0;0',
                        'Se;6;1;5;0;0', 'Si;4;4;0;0;0', 'V;8;5;0;0;1']

    PossibleECIF = [i[0]+"-"+i[1]
                    for i in product(ECIF_ProteinAtoms, ECIF_LigandAtoms)]

    atom_type_radii = {
        'As': 1.85,
        'B': 0.85,
        'Be': 1.53,
        'Br': 1.85,
        'C': 1.7,
        'Cl': 1.75,
        'Cu': 1.28,
        'F': 1.47,
        'Fe': 1.26,
        'Hg': 1.5,
        'I': 1.98,
        'Ir': 2.0,
        'Mg': 1.73,
        'N': 1.55,
        'O': 1.52,
        'Os': 2.0,
        'P': 1.8,
        'Pt': 1.75,
        'Re': 2.05,
        'Rh': 2.0,
        'Ru': 2.05,
        'S': 1.8,
        'Sb': 2.06,
        'Se': 1.9,
        'Si': 2.1,
        'V': 1.34
    }

    Atom_Keys = pd.read_csv(
        "../utils//PDB_Atom_Keys.csv", sep=",")

    def __init__(self, Kernel, cutoff):
        self.Kernel = Kernel
        self.cutoff = cutoff

    def GetAtomType(self, atom):
        # This function takes an atom in a molecule and returns its type as defined for ECIF

        AtomType = [atom.GetSymbol(),
                    str(atom.GetExplicitValence()),
                    str(len([x.GetSymbol()
                             for x in atom.GetNeighbors() if x.GetSymbol() != "H"])),
                    str(len([x.GetSymbol()
                             for x in atom.GetNeighbors() if x.GetSymbol() == "H"])),
                    str(int(atom.GetIsAromatic())),
                    str(int(atom.IsInRing())),
                    ]

        return(";".join(AtomType))

    def LoadSDFasDF(self, SDF):
        # This function takes an SDF for a ligand as input and returns it as a pandas DataFrame with its atom types labeled according to ECIF

        m = Chem.MolFromMolFile(SDF, sanitize=False)
        m.UpdatePropertyCache(strict=False)

        ECIF_atoms = []

        for atom in m.GetAtoms():
            if atom.GetSymbol() != "H":  # Include only non-hydrogen atoms
                entry = [int(atom.GetIdx())]
                entry.append(self.GetAtomType(atom))
                pos = m.GetConformer().GetAtomPosition(atom.GetIdx())
                entry.append(float("{0:.4f}".format(pos.x)))
                entry.append(float("{0:.4f}".format(pos.y)))
                entry.append(float("{0:.4f}".format(pos.z)))
                ECIF_atoms.append(entry)

        df = pd.DataFrame(ECIF_atoms)
        df.columns = ["ATOM_INDEX", "ECIF_ATOM_TYPE", "X", "Y", "Z"]
        if len(set(df["ECIF_ATOM_TYPE"]) - set(self.ECIF_LigandAtoms)) > 0:
            print(
                "WARNING: Ligand contains unsupported atom types. Only supported atom-type pairs are counted.")
            print(set(df["ECIF_ATOM_TYPE"]) - set(self.ECIF_LigandAtoms))
        return(df)

    def LoadPDBasDF(self, PDB):
        # This function takes a PDB for a protein as input and returns it as a pandas DataFrame with its atom types labeled according to ECIF

        ECIF_atoms = []

        f = open(PDB)
        for i in f:
            if i[:4] == "ATOM":
                # Include only non-hydrogen atoms
                if (
                    len(i[12:16].replace(" ", "")) < 4 and 
                    i[12:16].replace(" ", "")[0] != "H") or (len(i[12:16].replace(" ", "")) == 4 and 
                    i[12:16].replace(" ", "")[1] != "H" and 
                    i[12:16].replace(" ", "")[0] != "H"
                    ):

                    ECIF_atoms.append([int(i[6:11]),
                                       i[17:20]+"-"+i[12:16].replace(" ", ""),
                                       float(i[30:38]),
                                       float(i[38:46]),
                                       float(i[46:54])
                                       ])

        f.close()

        df = pd.DataFrame(ECIF_atoms, columns=[
                          "ATOM_INDEX", "PDB_ATOM", "X", "Y", "Z"])
        df = df.merge(self.Atom_Keys, left_on='PDB_ATOM', right_on='PDB_ATOM')[
            ["ATOM_INDEX", "ECIF_ATOM_TYPE", "X", "Y", "Z"]].sort_values(by="ATOM_INDEX").reset_index(drop=True)
        if list(df["ECIF_ATOM_TYPE"].isna()).count(True) > 0:
            print(
                "WARNING: Protein contains unsupported atom types. Only supported atom-type pairs are counted.")
        return(df)

    def get_mwcg_rigidity(self, protein_file, ligand_file):
        '''
            Adapted from ECIF package
        '''
        protein = self.LoadPDBasDF(protein_file)
        ligand = self.LoadSDFasDF(ligand_file)

        # select protein atoms in a cubic with a size of cutoff from ligand
        for i in ["X", "Y", "Z"]:
            protein = protein[protein[i] < float(ligand[i].max())+self.cutoff]
            protein = protein[protein[i] > float(ligand[i].min())-self.cutoff]

        atom_pairs = list(
            product(protein["ECIF_ATOM_TYPE"], ligand["ECIF_ATOM_TYPE"]))

        atom_pairs = [x[0]+"-"+x[1] for x in atom_pairs]
        atom_pairs = pd.DataFrame(atom_pairs, columns=["ATOM_PAIR"])
        atom_pairs["ELEMENTS_PAIR"] = [x.split(
            "-")[0].split(";")[0]+"-"+x.split("-")[1].split(";")[0] for x in atom_pairs["ATOM_PAIR"]]
        atom_pairs["PAIR_RADII"] = [self.atom_type_radii[x.split(
            "-")[0]] + self.atom_type_radii[x.split("-")[1]] for x in atom_pairs["ELEMENTS_PAIR"]]

        pairwise_radii = atom_pairs['PAIR_RADII'].values

        pairwise_mwcg = atom_pairs.drop(
            ["ELEMENTS_PAIR", "PAIR_RADII"], axis=1)

        distances = cdist(protein[["X", "Y", "Z"]],
                          ligand[["X", "Y", "Z"]], metric="euclidean")
        pairwise_radii = pairwise_radii.reshape(
            distances.shape[0], distances.shape[1])
        mwcg_distances = self.Kernel.kernel_function(distances, pairwise_radii)

        distances = distances.ravel()
        mwcg_distances = mwcg_distances.ravel()
        mwcg_distances = pd.DataFrame(
            data={"DISTANCE": distances, "MWCG_DISTANCE": mwcg_distances})
        pairwise_mwcg = pd.concat([pairwise_mwcg, mwcg_distances], axis=1)
        pairwise_mwcg = pairwise_mwcg[pairwise_mwcg["DISTANCE"] <= self.cutoff].reset_index(
            drop=True)

        return pairwise_mwcg

    def get_ggl_score(self, protein_file, ligand_file):
        features = ['COUNTS', 'SUM', 'MEAN', 'STD', 'MIN', 'MAX']
        pairwise_mwcg = self.get_mwcg_rigidity(protein_file, ligand_file)
        mwcg_temp_grouped = pairwise_mwcg.groupby('ATOM_PAIR')
        mwcg_temp_grouped.agg(['sum', 'mean', 'std', 'min', 'max'])
        mwcg_temp = mwcg_temp_grouped.size().to_frame(name='COUNTS')
        mwcg_temp = (mwcg_temp
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'sum'}).rename(columns={'MWCG_DISTANCE': 'SUM'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'mean'}).rename(columns={'MWCG_DISTANCE': 'MEAN'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'std'}).rename(columns={'MWCG_DISTANCE': 'STD'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'min'}).rename(columns={'MWCG_DISTANCE': 'MIN'}))
                     .join(mwcg_temp_grouped.agg({'MWCG_DISTANCE': 'max'}).rename(columns={'MWCG_DISTANCE': 'MAX'}))
                     )

        mwcg_columns = {'ATOM_PAIR': self.PossibleECIF}
        for _f in features:
            mwcg_columns[_f] = np.zeros(len(self.PossibleECIF))
        ggl_score = pd.DataFrame(data=mwcg_columns)
        ggl_score = ggl_score.set_index('ATOM_PAIR').add(
            mwcg_temp, fill_value=0).reindex(self.PossibleECIF).reset_index()

        return ggl_score

In [12]:
# Path for the protein pdb file and ligand sdf file
protein_pdbfile = 'Example_Complex/1a1e/1a1e_protein.pdb'
ligand_sdffile = 'Example_Complex/1a1e/1a1e_ligand.sdf'

In [13]:
# Construct the ECIF atom-type GGL class with the given Kernel and a cutoff distance
ecif_GGL = ECIF_GGL(Kernel, cutoff=12.0)

In [14]:
protein_df = ecif_GGL.LoadPDBasDF(protein_pdbfile)
protein_df.head()

,ATOM_INDEX,ECIF_ATOM_TYPE,X,Y,Z
0,1,N;3;2;1;0;0,57.904,24.527,16.458
1,5,C;4;3;1;0;0,56.635,24.239,17.114
2,7,C;4;3;0;0;0,56.572,24.562,18.605
3,8,O;2;1;0;0;0,55.469,24.718,19.110
4,9,C;4;3;1;0;0,56.298,22.729,16.823


In [15]:
ligand_df = ecif_GGL.LoadSDFasDF(ligand_sdffile)
ligand_df.head()

,ATOM_INDEX,ECIF_ATOM_TYPE,X,Y,Z
0,0,C;4;3;0;0;0,43.293,-8.128,37.550
1,1,O;2;1;0;0;0,44.067,-7.186,37.788
2,2,C;4;1;3;0;0,43.519,-9.010,36.332
3,3,N;3;2;1;0;0,42.169,-8.311,38.208
4,4,C;4;3;1;0;0,41.875,-7.504,39.354


In [16]:
# Pairwise protein-ligand ECIF atom type colored graph for the example structure
pairwise_mwcg = ecif_GGL.get_mwcg_rigidity(protein_pdbfile, ligand_sdffile)
pairwise_mwcg.head()

,ATOM_PAIR,DISTANCE,MWCG_DISTANCE
0,C;4;2;1;1;1-O;2;1;0;0;0,11.748670,0.562132
1,C;4;3;0;1;1-O;2;1;0;0;0,11.963535,0.526140
2,O;2;1;1;0;0-O;2;1;0;0;0,11.370757,0.512450
3,C;4;3;0;0;0-O;2;1;0;0;0,11.510658,0.600823
4,O;2;1;0;0;0-O;2;1;0;0;0,11.338683,0.518240


In [17]:
# ECIF-GGL features for the example structure
ecif_ggl_score = ecif_GGL.get_ggl_score(protein_pdbfile, ligand_sdffile)
ecif_ggl_score.head(40)

,ATOM_PAIR,COUNTS,SUM,MEAN,STD,MIN,MAX
0,C;4;1;3;0;0-As;1;1;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,C;4;1;3;0;0-As;4;4;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,C;4;1;3;0;0-B;1;1;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,C;4;1;3;0;0-B;2;2;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,C;4;1;3;0;0-B;3;2;1;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,C;4;1;3;0;0-B;3;3;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,C;4;1;3;0;0-B;3;3;0;0;1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,C;4;1;3;0;0-B;3;4;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,C;4;1;3;0;0-B;3;4;0;0;1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,C;4;1;3;0;0-B;4;4;0;0;0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
